In [1]:
# Load necessary packages
library(RNifti)
library(dplyr)
library(oro.nifti)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


oro.nifti 0.11.4


Attaching package: ‘oro.nifti’


The following object is masked from ‘package:dplyr’:

    slice


The following object is masked from ‘package:RNifti’:

    origin




In [2]:
# Create necessary data frames
results_dir <- "scratch/tractoflow_hcp_dwi/results/"
patients <- list.files(results_dir, full.names = TRUE)
patients <- patients[grep("^\\d{6}$", basename(patients))]
patient_data <- read.csv("scratch/tractoflow_hcp_dwi/patient_data.csv")
patient_data_gender <- read.csv("scratch/tractoflow_hcp_dwi/patient_data_gender.csv")
subj_data <- patient_data[c("Subject", "Age_in_Yrs", "BMI")]
subj_data_gender <- patient_data_gender[c("Subject", "Gender")]
subj_data=merge(subj_data, subj_data_gender)

# Define variable to extract hypothalamic values
rois <- c("md_anterior_hypothalamic_area", "md_arcuate_hypothalamic_nucleus", "md_dorsal_periventricular_hypothalamic_nucleus", "md_dorsomedial_hypothalamic_nucleus", "md_lateral_hypothalamus", "md_mammillary_bodies", "md_medial_preoptic_nucleus", "md_paraventricular_nucleus", "md_periventricular_hypothalamic_nucleus", "md_posterior_hypothalamic_nucleus", "md_suprachiasmatic_hypothalamic_nucleus", "md_supraoptic_hypothalamic_nucleus", "md_tuberomammillary_hypothalamic_nucleus", "md_ventromedial_hypothalamus")
rois_all <- c("md_anterior_hypothalamic_area_lh", "md_arcuate_hypothalamic_nucleus_lh", "md_dorsal_periventricular_hypothalamic_nucleus_lh", "md_dorsomedial_hypothalamic_nucleus_lh", "md_lateral_hypothalamus_lh", "md_mammillary_bodies_lh", "md_medial_preoptic_nucleus_lh", "md_paraventricular_nucleus_lh", "md_periventricular_hypothalamic_nucleus_lh", "md_posterior_hypothalamic_nucleus_lh", "md_suprachiasmatic_hypothalamic_nucleus_lh", "md_supraoptic_hypothalamic_nucleus_lh", "md_tuberomammillary_hypothalamic_nucleus_lh", "md_ventromedial_hypothalamus_lh","md_anterior_hypothalamic_area_rh", "md_arcuate_hypothalamic_nucleus_rh", "md_dorsal_periventricular_hypothalamic_nucleus_rh", "md_dorsomedial_hypothalamic_nucleus_rh", "md_lateral_hypothalamus_rh", "md_mammillary_bodies_rh", "md_medial_preoptic_nucleus_rh", "md_paraventricular_nucleus_rh", "md_periventricular_hypothalamic_nucleus_rh", "md_posterior_hypothalamic_nucleus_rh", "md_suprachiasmatic_hypothalamic_nucleus_rh", "md_supraoptic_hypothalamic_nucleus_rh", "md_tuberomammillary_hypothalamic_nucleus_rh", "md_ventromedial_hypothalamus_rh")

In [3]:
# Create empty data frame
roi_data <- data.frame(Subject=character(0))

# Loop through each patient
for (i in 1:length(patients)) {
    # Define necessary lists
    var_names_lh <-list()
    var_names_rh <-list()
    data_lh <-list()
    data_rh <-list()
    avg_vals <- list()
    
    # Loop through each ROI
    for (roi in 1:length(rois)) {
        # Predefining the first column values
        if (i==1) {
            roi_data[[rois[roi]]] <- numeric(0)
        }
        # Define paths to left and right hemisphere of ROI
        var_names_lh[[roi]] <- file.path("scratch", "tractoflow_hcp_dwi", "subject_rois_bin", basename(patients[i]), "md_rois", paste0(basename(patients[i]), "_", rois[roi], "_lh_resampled.nii.gz"))
        var_names_rh[[roi]] <- file.path("scratch", "tractoflow_hcp_dwi", "subject_rois_bin", basename(patients[i]), "md_rois", paste0(basename(patients[i]), "_", rois[roi], "_rh_resampled.nii.gz"))
        # Read the NIfTI file paths
        data_lh[[roi]] <- readNifti(var_names_lh[[roi]])
        data_rh[[roi]] <- readNifti(var_names_rh[[roi]])
        # Define and extract all values above 0 of the NIfTI ROI
        non_zero_vals_lh <- data_lh[[roi]]
        non_zero_vals_rh <- data_rh[[roi]]
        non_zero_vals_lh <- non_zero_vals_lh[non_zero_vals_lh > 0]
        non_zero_vals_rh <- non_zero_vals_rh[non_zero_vals_rh > 0]
        # Calculate ROI average value and append it to the data frame
        avg_val <- (mean(non_zero_vals_lh)+mean(non_zero_vals_rh))/2
        avg_vals <- append(avg_vals, avg_val)
        temp_data <- data.frame(Subject=basename(patients[i]), avg_vals)
    }
    # Define column names and bind data calculated in the for loop
    colnames(temp_data)=c("Subject", rois)
    roi_data <- rbind(roi_data, temp_data)
}
# Merge hypothalamic values with patient data and write as CSV
subj_data <- merge(subj_data, roi_data)
write.csv(subj_data, "scratch/tractoflow_hcp_dwi/linear_regression/HCP_md_data_bin.csv")